# Manufacturing: Delta Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a manufacturing analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Production Quality Control and Equipment Monitoring

We'll analyze manufacturing production records from a factory. Our clustering strategy will optimize for:

- **Equipment-specific queries**: Fast lookups by machine ID
- **Time-based analysis**: Efficient filtering by production date
- **Quality control patterns**: Quick aggregation by product type and defect rates

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create manufacturing catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS manufacturing")

spark.sql("CREATE SCHEMA IF NOT EXISTS manufacturing.analytics")

print("Manufacturing catalog and analytics schema created successfully!")

Manufacturing catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `production_records` table will store:

- **machine_id**: Unique equipment identifier
- **production_date**: Date and time of production
- **product_type**: Type of product manufactured
- **units_produced**: Number of units produced
- **defect_count**: Number of defective units
- **production_line**: Assembly line identifier
- **cycle_time**: Time to produce one unit (minutes)

### Clustering Strategy

We'll cluster by `machine_id` and `production_date` because:

- **machine_id**: Equipment often produces multiple batches, grouping maintenance and performance data together
- **production_date**: Time-based queries are essential for shift analysis, maintenance scheduling, and quality trending
- This combination optimizes for both equipment monitoring and temporal production analysis

In [ ]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS manufacturing.analytics.production_records (

    machine_id STRING,

    production_date TIMESTAMP,

    product_type STRING,

    units_produced INT,

    defect_count INT,

    production_line STRING,

    cycle_time DECIMAL(5,2)

)

USING DELTA

CLUSTER BY (machine_id, production_date)

""")

print("Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on machine_id and production_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on machine_id and production_date.


## Step 3: Generate Manufacturing Sample Data

### Data Generation Strategy

We'll create realistic manufacturing production data including:

- **200 machines** with multiple production runs over time
- **Product types**: Electronics, Automotive Parts, Consumer Goods, Industrial Equipment
- **Realistic production patterns**: Shift-based operations, maintenance downtime, quality variations
- **Multiple production lines**: Different assembly areas and facilities

### Why This Data Pattern?

This data simulates real manufacturing scenarios where:

- Equipment performance varies over time
- Quality control requires tracking defects and yields
- Maintenance scheduling depends on usage patterns
- Production optimization drives efficiency improvements
- Supply chain visibility requires real-time production data

In [ ]:
# Generate sample manufacturing production data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define manufacturing data constants

PRODUCT_TYPES = ['Electronics', 'Automotive Parts', 'Consumer Goods', 'Industrial Equipment']

PRODUCTION_LINES = ['LINE_A', 'LINE_B', 'LINE_C', 'LINE_D', 'LINE_E']

# Base production parameters by product type

PRODUCTION_PARAMS = {

    'Electronics': {'base_units': 500, 'defect_rate': 0.02, 'cycle_time': 2.5},

    'Automotive Parts': {'base_units': 200, 'defect_rate': 0.05, 'cycle_time': 8.0},

    'Consumer Goods': {'base_units': 800, 'defect_rate': 0.03, 'cycle_time': 1.8},

    'Industrial Equipment': {'base_units': 50, 'defect_rate': 0.08, 'cycle_time': 25.0}

}


# Generate production records

production_data = []

base_date = datetime(2024, 1, 1)


# Create 200 machines with 30-90 production runs each

for machine_num in range(1, 201):

    machine_id = f"MCH{machine_num:04d}"
    
    # Each machine gets 30-90 production runs over 12 months

    num_runs = random.randint(30, 90)
    
    for i in range(num_runs):

        # Spread production runs over 12 months (weekdays only, during shifts)

        days_offset = random.randint(0, 365)

        production_date = base_date + timedelta(days=days_offset)
        
        # Skip weekends

        while production_date.weekday() >= 5:

            production_date += timedelta(days=1)
        
        # Add shift timing (6 AM - 6 PM)

        hours_offset = random.randint(6, 18)

        production_date = production_date.replace(hour=hours_offset, minute=0, second=0, microsecond=0)
        
        # Select product type

        product_type = random.choice(PRODUCT_TYPES)

        params = PRODUCTION_PARAMS[product_type]
        
        # Calculate production with variability

        units_variation = random.uniform(0.7, 1.3)

        units_produced = int(params['base_units'] * units_variation)
        
        # Calculate defects

        defect_rate_variation = random.uniform(0.5, 2.0)

        actual_defect_rate = params['defect_rate'] * defect_rate_variation

        defect_count = int(units_produced * actual_defect_rate)
        
        # Calculate cycle time with variation

        cycle_time_variation = random.uniform(0.8, 1.4)

        cycle_time = round(params['cycle_time'] * cycle_time_variation, 2)
        
        # Select production line

        production_line = random.choice(PRODUCTION_LINES)
        
        production_data.append({

            "machine_id": machine_id,

            "production_date": production_date,

            "product_type": product_type,

            "units_produced": units_produced,

            "defect_count": defect_count,

            "production_line": production_line,

            "cycle_time": cycle_time

        })



print(f"Generated {len(production_data)} production records")

print("Sample record:", production_data[0])

Generated 12298 production records
Sample record: {'machine_id': 'MCH0001', 'production_date': datetime.datetime(2024, 9, 6, 6, 0), 'product_type': 'Industrial Equipment', 'units_produced': 36, 'defect_count': 2, 'production_line': 'LINE_A', 'cycle_time': 22.15}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_production = spark.createDataFrame(production_data)


# Display schema and sample data

print("DataFrame Schema:")

df_production.printSchema()



print("\nSample Data:")

df_production.show(5)


# Insert data into Delta table with liquid clustering

# The CLUSTER BY (machine_id, production_date) will automatically optimize the data layout

df_production.write.mode("overwrite").saveAsTable("manufacturing.analytics.production_records")


print(f"\nSuccessfully inserted {df_production.count()} records into manufacturing.analytics.production_records")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- cycle_time: double (nullable = true)
 |-- defect_count: long (nullable = true)
 |-- machine_id: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- production_date: timestamp (nullable = true)
 |-- production_line: string (nullable = true)
 |-- units_produced: long (nullable = true)


Sample Data:


+----------+------------+----------+--------------------+-------------------+---------------+--------------+
|cycle_time|defect_count|machine_id|        product_type|    production_date|production_line|units_produced|
+----------+------------+----------+--------------------+-------------------+---------------+--------------+
|     22.15|           2|   MCH0001|Industrial Equipment|2024-09-06 06:00:00|         LINE_A|            36|
|      1.75|          30|   MCH0001|      Consumer Goods|2024-03-26 11:00:00|         LINE_B|          1034|
|      9.09|           8|   MCH0001|    Automotive Parts|2024-12-30 17:00:00|         LINE_B|           259|
|      2.65|          25|   MCH0001|         Electronics|2024-10-21 06:00:00|         LINE_B|           641|
|      2.11|           9|   MCH0001|         Electronics|2024-05-13 18:00:00|         LINE_A|           437|
+----------+------------+----------+--------------------+-------------------+---------------+--------------+
only showing top 5 


Successfully inserted 12298 records into manufacturing.analytics.production_records
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Machine performance history** (clustered by machine_id)
2. **Time-based production analysis** (clustered by production_date)
3. **Combined machine + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Machine performance history - benefits from machine_id clustering

print("=== Query 1: Machine Performance History ===")

machine_history = spark.sql("""

SELECT machine_id, production_date, product_type, units_produced, defect_count,

       ROUND(defect_count * 100.0 / units_produced, 2) as defect_rate_percent

FROM manufacturing.analytics.production_records

WHERE machine_id = 'MCH0001'

ORDER BY production_date DESC

""")



machine_history.show()

print(f"Records found: {machine_history.count()}")



# Query 2: Time-based quality analysis - benefits from production_date clustering

print("\n=== Query 2: Recent Quality Issues ===")

quality_issues = spark.sql("""

SELECT production_date, machine_id, product_type, units_produced, defect_count,

       ROUND(defect_count * 100.0 / units_produced, 2) as defect_rate_percent

FROM manufacturing.analytics.production_records

WHERE production_date >= '2024-06-01' AND (defect_count * 100.0 / units_produced) > 5.0

ORDER BY defect_rate_percent DESC, production_date DESC

""")



quality_issues.show()

print(f"Quality issues found: {quality_issues.count()}")



# Query 3: Combined machine + time query - optimal for our clustering strategy

print("\n=== Query 3: Equipment Performance Trends ===")

performance_trends = spark.sql("""

SELECT machine_id, production_date, product_type, units_produced, cycle_time,

       ROUND(units_produced * 60.0 / cycle_time, 2) as hourly_rate

FROM manufacturing.analytics.production_records

WHERE machine_id LIKE 'MCH000%' AND production_date >= '2024-04-01'

ORDER BY machine_id, production_date

""")



performance_trends.show()

print(f"Performance records found: {performance_trends.count()}")

=== Query 1: Machine Performance History ===


+----------+-------------------+--------------------+--------------+------------+-------------------+
|machine_id|    production_date|        product_type|units_produced|defect_count|defect_rate_percent|
+----------+-------------------+--------------------+--------------+------------+-------------------+
|   MCH0001|2024-12-30 17:00:00|    Automotive Parts|           259|           8|               3.09|
|   MCH0001|2024-12-27 08:00:00|Industrial Equipment|            58|           5|               8.62|
|   MCH0001|2024-12-23 08:00:00|Industrial Equipment|            45|           5|              11.11|
|   MCH0001|2024-12-17 08:00:00|         Electronics|           647|          22|               3.40|
|   MCH0001|2024-12-11 14:00:00|Industrial Equipment|            48|           5|              10.42|
|   MCH0001|2024-12-02 18:00:00|    Automotive Parts|           175|          14|               8.00|
|   MCH0001|2024-12-02 08:00:00|    Automotive Parts|           184|           4| 

Records found: 64

=== Query 2: Recent Quality Issues ===


+-------------------+----------+--------------------+--------------+------------+-------------------+
|    production_date|machine_id|        product_type|units_produced|defect_count|defect_rate_percent|
+-------------------+----------+--------------------+--------------+------------+-------------------+
|2024-07-26 06:00:00|   MCH0183|Industrial Equipment|            44|           7|              15.91|
|2024-11-04 15:00:00|   MCH0135|Industrial Equipment|            63|          10|              15.87|
|2024-12-13 16:00:00|   MCH0169|Industrial Equipment|            51|           8|              15.69|
|2024-10-25 11:00:00|   MCH0023|Industrial Equipment|            51|           8|              15.69|
|2024-09-04 08:00:00|   MCH0086|Industrial Equipment|            51|           8|              15.69|
|2024-09-03 16:00:00|   MCH0001|Industrial Equipment|            64|          10|              15.63|
|2024-11-28 12:00:00|   MCH0099|Industrial Equipment|            45|           7| 

Quality issues found: 2950

=== Query 3: Equipment Performance Trends ===


+----------+-------------------+--------------------+--------------+----------+-----------+
|machine_id|    production_date|        product_type|units_produced|cycle_time|hourly_rate|
+----------+-------------------+--------------------+--------------+----------+-----------+
|   MCH0001|2024-04-01 13:00:00|    Automotive Parts|           204|     10.15|    1205.91|
|   MCH0001|2024-04-01 15:00:00|    Automotive Parts|           214|      9.77|    1314.23|
|   MCH0001|2024-04-11 09:00:00|         Electronics|           613|       3.3|   11145.45|
|   MCH0001|2024-04-29 09:00:00|Industrial Equipment|            49|     25.46|     115.48|
|   MCH0001|2024-04-30 09:00:00|    Automotive Parts|           209|       6.8|    1844.12|
|   MCH0001|2024-05-07 13:00:00|Industrial Equipment|            47|     33.01|      85.43|
|   MCH0001|2024-05-13 18:00:00|         Electronics|           437|      2.11|   12426.54|
|   MCH0001|2024-05-14 18:00:00|Industrial Equipment|            44|     30.47| 

Performance records found: 382


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the manufacturing insights possible with this optimized structure.

### Key Analytics

- **Equipment utilization** and performance metrics
- **Quality control analysis** and defect patterns
- **Production line efficiency** and bottleneck identification
- **Product type performance** and optimization opportunities

In [ ]:
# Analyze clustering effectiveness and manufacturing insights


# Equipment performance analysis

print("=== Equipment Performance Analysis ===")

equipment_performance = spark.sql("""

SELECT machine_id, COUNT(*) as total_runs,

       ROUND(AVG(units_produced), 2) as avg_units_produced,

       ROUND(AVG(defect_count * 100.0 / units_produced), 2) as avg_defect_rate,

       ROUND(AVG(cycle_time), 2) as avg_cycle_time,

       ROUND(SUM(units_produced), 0) as total_units

FROM manufacturing.analytics.production_records

GROUP BY machine_id

ORDER BY total_units DESC

""")



equipment_performance.show()


# Quality analysis by product type

print("\n=== Quality Analysis by Product Type ===")

quality_by_product = spark.sql("""

SELECT product_type, COUNT(*) as production_runs,

       ROUND(SUM(units_produced), 0) as total_units,

       ROUND(SUM(defect_count), 0) as total_defects,

       ROUND(AVG(defect_count * 100.0 / units_produced), 2) as avg_defect_rate,

       ROUND(AVG(cycle_time), 2) as avg_cycle_time

FROM manufacturing.analytics.production_records

GROUP BY product_type

ORDER BY total_units DESC

""")



quality_by_product.show()


# Production line efficiency

print("\n=== Production Line Efficiency ===")

line_efficiency = spark.sql("""

SELECT production_line, COUNT(*) as total_runs,

       COUNT(DISTINCT machine_id) as machines_used,

       ROUND(SUM(units_produced), 0) as total_production,

       ROUND(AVG(units_produced), 2) as avg_run_size,

       ROUND(SUM(defect_count * 100.0 / units_produced) / COUNT(*), 2) as avg_defect_rate

FROM manufacturing.analytics.production_records

GROUP BY production_line

ORDER BY total_production DESC

""")



line_efficiency.show()


# Monthly production trends

print("\n=== Monthly Production Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(production_date, 'yyyy-MM') as month,

       COUNT(*) as production_runs,

       ROUND(SUM(units_produced), 0) as total_units,

       ROUND(AVG(defect_count * 100.0 / units_produced), 2) as avg_defect_rate,

       COUNT(DISTINCT machine_id) as active_machines

FROM manufacturing.analytics.production_records

GROUP BY DATE_FORMAT(production_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Equipment Performance Analysis ===


+----------+----------+------------------+---------------+--------------+-----------+
|machine_id|total_runs|avg_units_produced|avg_defect_rate|avg_cycle_time|total_units|
+----------+----------+------------------+---------------+--------------+-----------+
|   MCH0163|        87|            437.47|           5.52|          9.06|      38060|
|   MCH0169|        83|            447.94|           5.18|         10.47|      37179|
|   MCH0006|        84|             432.4|           4.73|          8.64|      36322|
|   MCH0108|        88|            411.07|           5.20|          8.93|      36174|
|   MCH0153|        88|            409.22|           5.03|         10.13|      36011|
|   MCH0097|        90|            396.48|           5.16|         10.12|      35683|
|   MCH0101|        86|            402.84|           4.81|          9.15|      34644|
|   MCH0070|        86|            402.67|           4.90|          9.46|      34630|
|   MCH0044|        84|            410.06|           5

+--------------------+---------------+-----------+-------------+---------------+--------------+
|        product_type|production_runs|total_units|total_defects|avg_defect_rate|avg_cycle_time|
+--------------------+---------------+-----------+-------------+---------------+--------------+
|      Consumer Goods|           2982|    2392057|        87950|           3.68|          1.98|
|         Electronics|           3172|    1580318|        37855|           2.39|          2.75|
|    Automotive Parts|           3091|     615989|        37171|           6.02|          8.82|
|Industrial Equipment|           3053|     151345|        13680|           8.99|         27.52|
+--------------------+---------------+-----------+-------------+---------------+--------------+


=== Production Line Efficiency ===


+---------------+----------+-------------+----------------+------------+---------------+
|production_line|total_runs|machines_used|total_production|avg_run_size|avg_defect_rate|
+---------------+----------+-------------+----------------+------------+---------------+
|         LINE_E|      2478|          200|          964486|      389.22|           5.18|
|         LINE_C|      2442|          200|          959370|      392.86|           5.19|
|         LINE_D|      2464|          200|          944868|      383.47|           5.27|
|         LINE_B|      2473|          200|          944805|      382.05|           5.28|
|         LINE_A|      2441|          200|          926180|      379.43|           5.35|
+---------------+----------+-------------+----------------+------------+---------------+


=== Monthly Production Trends ===


+-------+---------------+-----------+---------------+---------------+
|  month|production_runs|total_units|avg_defect_rate|active_machines|
+-------+---------------+-----------+---------------+---------------+
|2024-01|           1045|     397326|           5.24|            198|
|2024-02|            914|     368483|           5.17|            195|
|2024-03|           1003|     383685|           5.41|            197|
|2024-04|           1074|     407309|           5.18|            199|
|2024-05|           1083|     413054|           5.38|            195|
|2024-06|            992|     375035|           5.28|            198|
|2024-07|           1138|     456635|           5.20|            197|
|2024-08|            930|     366966|           5.01|            195|
|2024-09|           1045|     391363|           5.28|            195|
|2024-10|           1015|     394063|           5.27|            198|
|2024-11|            946|     363835|           5.30|            192|
|2024-12|           

## Key Takeaways: Delta Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (machine_id, production_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (machine_id, production_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Manufacturing analytics where equipment monitoring and quality control are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for manufacturing data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles manufacturing-scale data volumes effortlessly

### Best Practices for Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger manufacturing datasets
- Integrate with real SCADA systems and IoT sensors

This notebook demonstrates how Oracle AI Data Platform makes advanced manufacturing analytics accessible while maintaining enterprise-grade performance and governance.